In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
df = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv')
df_test = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')

In [3]:
df.head()

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,...,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
1,007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,...,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
2,013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,...,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0
3,043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,...,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0
4,044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,...,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1


In [4]:
from sklearn import preprocessing
  
# label_encoder object knows 
# how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
df['EJ']= label_encoder.fit_transform(df['EJ'])
# Encode labels in column 'species'.
#df = pd.get_dummies(df, columns = ['EJ'])

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
from sklearn.impute import KNNImputer
imp = KNNImputer()
df["GL"] = imp.fit_transform(df[["GL"]]).ravel()
df["FS"] = imp.fit_transform(df[["FS"]]).ravel()
df["FL"] = imp.fit_transform(df[["FL"]]).ravel()
df["FC"] = imp.fit_transform(df[["FC"]]).ravel()
df["EL"] = imp.fit_transform(df[["EL"]]).ravel()
df["DU"] = imp.fit_transform(df[["DU"]]).ravel()
df["CB"] = imp.fit_transform(df[["CB"]]).ravel()
df["CC"] = imp.fit_transform(df[["CC"]]).ravel()
df["BQ"] = imp.fit_transform(df[["BQ"]]).ravel()


In [6]:
X = df.loc[:, df.columns != 'Class']
y = df.loc[:, df.columns == 'Class']
X=X.drop(['Id'],axis=1)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [7]:
# from sklearn.linear_model import LogisticRegression
# reg = LogisticRegression(solver='lbfgs',max_iter=3000)
# reg.fit(X_train, y_train.values.ravel())
# from sklearn.tree import DecisionTreeClassifier
# dec_tree = DecisionTreeClassifier()
# dec_tree.fit(X_train,y_train.values.ravel())
import xgboost as xgb
xgb= xgb.XGBClassifier(objective="binary:logistic")
xgb.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [8]:
df_test=df_test.drop(['Id'],axis=1)
df_test['EJ']= label_encoder.fit_transform(df_test['EJ'])
#df_test = pd.get_dummies(df_test, columns = ['EJ'])
#df_test['EJ_B']=0


In [9]:
# df_test = df_test.drop(df_test.columns[unimp],axis = 1)

In [10]:
preds=xgb.predict_proba(df_test)

In [11]:
preds

array([[0.75388896, 0.24611104],
       [0.75388896, 0.24611104],
       [0.75388896, 0.24611104],
       [0.75388896, 0.24611104],
       [0.75388896, 0.24611104]], dtype=float32)

In [12]:
# l=list(xgb.feature_importances_)
# unimp=[]
# for i in range(0,len(l)):
#     if l[i]<0.01:
#         unimp.append(i)
# print(unimp)

In [13]:
#df = df.drop(df.columns[unimp],axis = 1)

In [14]:
sub=pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv')
sub[['class_0','class_1']]=preds
sub.to_csv('/kaggle/working/submission.csv',index=False)